In [3]:
import sys
import os
sys.path.append("./../")
sys.path.append("./../external")
import numpy as np
import time
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt
from pyminion import *

from external.implementations.j2020 import j2020 
#from external.implementations.esCMAgES import esCMAgES 
#from external.implementations.AGSK import AGSK
#from external.implementations.COLSHADE import COLSHADE

In [26]:
import numpy as np
from scipy.optimize import OptimizeResult

class MiniSHADE:
    """
    Memory-based DE (MADE) optimization algorithm implementation
    """
    def __init__(self, func, bounds, args=(), population_size=20, strategy='current_to_pbest1bin', memorySize=10, minPopSize=10, seed=None):
        self.func = func
        self.funcArgs = args
        self.bounds = np.array(bounds, dtype=np.float64)
        self.dim = len(bounds)
        self.popsize = population_size
        self.strategy = strategy
        self.H = memorySize  # Memory size for CR and F
        self.minPopSize = minPopSize
        if seed is not None:
            np.random.seed(seed)
        self._initialize_population()
        self.hasInitialized = False

    def _initialize_population(self):
        self.M_CR = np.random.uniform(0.4, 0.6, self.H)
        self.M_F = np.random.uniform(0.6, 1.0, self.H)
        self.F = np.full(self.H, 0.8)
        self.CR = np.random.uniform(0.8, 1.0, self.popsize)
        self.hasInitialized = True
        self.population = np.random.uniform(self.bounds[:, 0], self.bounds[:, 1], (self.popsize, self.dim))
        self.fitness = self.func(self.population, *self.funcArgs)
        self.best_idx = np.argmin(self.fitness)
        self.best = self.population[self.best_idx]
        self.best_fitness = self.fitness[self.best_idx]
        self.fitnessHistory = [self.best_fitness]
        self.muCR, self.stdCR = [], []
        self.muF, self.stdF = [], []

    def _adapt_parameters(self):
        idx = np.random.randint(0, self.H, self.popsize)
        self.CR = np.random.normal(self.M_CR[idx], 0.1)
        self.F = np.random.normal(self.M_F[idx], 0.1)
        self.CR = np.clip(self.CR, 0.01, 1.0)
        self.F = np.clip(self.F, 0.01, 2.0)
        self.muCR.append(np.mean(self.CR))
        self.muF.append(np.mean(self.F))
        self.stdCR.append(np.std(self.CR))
        self.stdF.append(np.std(self.F))

    def _mutate(self, idx):
        available_indices = np.delete(np.arange(self.popsize), idx)
        frac = int(0.2*self.popsize)
        if frac <= 2 : p=2
        else : p = np.random.choice([1+i for i in range(frac)])
        pbestind= np.random.choice(np.argsort(self.fitness)[:p])
        r1, r2 = np.random.choice(available_indices, 2, replace=False)
        mutant = self.population[idx] + self.F[idx] * (self.population[pbestind] - self.population[idx]) + self.F[idx] * (self.population[r1] - self.population[r2])
        return mutant

    def _crossover(self, target, mutant, CR):
        crossover_mask = np.random.rand(self.dim) < CR
        if not np.any(crossover_mask):
            crossover_mask[np.random.randint(0, self.dim)] = True
        return np.where(crossover_mask, mutant, target)

    def enforce_bounds(self, new_candidates):
        lower_bounds, upper_bounds = self.bounds.T
        np.clip(new_candidates, lower_bounds, upper_bounds, out=new_candidates)

    def evolve(self):
        if not self.hasInitialized:
            self._initialize_population()
        self._adapt_parameters()
        new_population = np.empty_like(self.population)
        new_fitness = np.empty_like(self.fitness)
        S_CR = []
        S_F = []
        weights = []
        weights_Lehmer = []

        all_trials = np.empty_like(self.population)
        for i in range(self.popsize):
            mutant = self._mutate(i)
            trial = self._crossover(self.population[i], mutant, self.CR[i])
            all_trials[i] = trial

        self.enforce_bounds(all_trials)
        all_trial_fitness = self.func(all_trials, *self.funcArgs)
        
        for i in range(self.popsize):
            trial_fitness = all_trial_fitness[i]
            if trial_fitness < self.fitness[i]:
                new_population[i] = all_trials[i]
                new_fitness[i] = trial_fitness
                S_CR.append(self.CR[i])
                S_F.append(self.F[i])
                w = (self.fitness[i] - trial_fitness) / (1e-100 + self.fitness[i])
                weights.append(w)
            else:
                new_population[i] = self.population[i]
                new_fitness[i] = self.fitness[i]

        self.population = new_population
        self.fitness = new_fitness
        self.best_idx = np.argmin(self.fitness)
        self.best = self.population[self.best_idx]
        self.best_fitness = self.fitness[self.best_idx]
        self.fitnessHistory.append(self.best_fitness)

        if len(S_CR) != 0:
            weights = np.array(weights)
            S_CR = np.array(S_CR)
            S_F = np.array(S_F)
            weights_Lehmer = (S_F * weights) / np.sum(S_F * weights)
            weights = np.array(weights) / np.sum(weights)
            muCR, stdCR = self.getMeanStd(S_CR, weights)
            muF, stdF = self.getMeanStd(S_F, weights_Lehmer)
            self.M_CR = np.random.choice(np.concatenate((self.M_CR, np.random.normal(muCR, stdCR, len(S_CR)))), self.H, replace=False)
            self.M_F = np.random.choice(np.concatenate((self.M_F, np.random.normal(muF, stdF, len(S_F)))), self.H, replace=False)

    def getMeanStd(self, arr, weight):
        arr = np.array(arr)
        weight = np.array(weight)
        weight = weight / np.sum(weight)
        mean = np.average(arr, weights=weight)
        variance = np.average((arr - mean) ** 2, weights=weight)
        std = np.sqrt(variance)
        return mean, std


class tDE:
    def __init__(self, func, bounds, args=(), x0=None, population_size=20, maxevals=1000000, strategy='pbest1bin', tol=0.0001, seed=None, Ntribes=5):
        self.func = func
        self.funcArgs = args
        self.bounds = np.array(bounds, dtype=np.float64)
        self.dim = len(bounds)
        self.maxevals = maxevals
        self.popsize = population_size
        self.Nevals = 0
        self.tol = tol
        self.strategy = strategy
        self.Ntribes = Ntribes
        self.use_clip = False
        if seed is not None:
            np.random.seed(seed)
        self.x0 = x0
        if self.x0 is not None:
            if len(self.x0) != self.dim:
                raise ValueError("x0 must have the same dimension as the length of the bounds.")
        self.tribes = self._initialize_tribes()

    def _initialize_tribes(self):
        tribes = []
        tribe_size = self.popsize // self.Ntribes
        for i in range(self.Ntribes):
            miniSHADE = MiniSHADE(
                func=self.func,
                bounds=self.bounds,
                args=self.funcArgs,
                population_size=tribe_size,
                strategy=self.strategy,
                memorySize=50,
                minPopSize=self.popsize,
            )
            miniSHADE._initialize_population()
            tribes.append(miniSHADE)
        return tribes

    def _marriage_probability(self):
        prob_matrix = np.zeros((self.Ntribes, self.Ntribes))
        centers = [np.mean(tribe.population, axis=0) for tribe in self.tribes]
        for i in range(self.Ntribes):
            distances = [np.linalg.norm(centers[i] - centers[j]) for j in range(self.Ntribes) if j != i]
            exp_distances = 1/(1e-10+np.array(distances))
            prob_matrix[i, [j for j in range(self.Ntribes) if j != i]] = exp_distances
            prob_matrix[i] /= np.sum(exp_distances)
        return 0.5*prob_matrix

    def _marriage(self, prob_matrix):
        # Create pairs of tribes first
        tribe_pairs = []
        for i in range(self.Ntribes):
            for j in range(i + 1, self.Ntribes):
                tribe_pairs.append((i, j))

        # Perform marriages for each pair based on the probabilities
        for (i, j) in tribe_pairs:
            if np.random.rand() < prob_matrix[i, j]:
                tribe1 = self.tribes[i]
                tribe2 = self.tribes[j]
                tribe_size = max(len(tribe1.population), len(tribe2.population))
                n_marriages = int(tribe_size)

                marriage_indices1 = np.random.choice(len(tribe1.population), n_marriages, replace=False)
                marriage_indices2 = np.random.choice(len(tribe2.population), n_marriages, replace=False)

                for idx1, idx2 in zip(marriage_indices1, marriage_indices2):
                    parent1_population = tribe1.population
                    parent1_fitness = tribe1.fitness
                    parent2_population = tribe2.population
                    parent2_fitness = tribe2.fitness

                    parent1_idx = np.random.randint(len(parent1_population))
                    parent2_idx = np.random.randint(len(parent2_population))

                    child1 = self._create_child(parent1_population[parent1_idx], parent2_population[parent2_idx], parent2_fitness[parent2_idx])
                    child2 = self._create_child(parent2_population[parent1_idx], parent2_population[parent2_idx], parent2_fitness[parent2_idx])

                    tribe1.population[idx1] = child1
                    tribe2.population[idx2] = child2


    def _create_child(self, A, B, fitness_B):
        child = np.empty_like(A)
        for i in range(len(A)):
            method = np.random.choice(['A', 'B', 'average'])
            if method == 'A':
                child[i] = A[i]
            elif method == 'B':
                child[i] = B[i]
            else:
                weight = 1 / fitness_B
                child[i] = (A[i] + weight * B[i]) / (1 + weight)
        return child

    def _reinitialize_tribes(self):
        for tribe in self.tribes:
                if np.std(tribe.fitness) < 0.01 * tribe.best_fitness:
                    best_individual = tribe.best
                    new_population = np.random.uniform(
                        self.bounds[:, 0], 
                        self.bounds[:, 1], 
                        (len(tribe.population), self.dim)
                    )
                    new_fitness = self.func(new_population, *self.funcArgs)
                    new_best_idx = np.argmin(new_fitness)
                    new_best = new_population[new_best_idx]
                    new_best_fitness = new_fitness[new_best_idx]

                    # Replace the worst individual with the best individual from the previous generation
                    worst_idx = np.argmax(new_fitness)
                    new_population[worst_idx] = best_individual
                    new_fitness[worst_idx] = tribe.best_fitness

                    tribe.population = new_population
                    tribe.fitness = new_fitness
                    tribe.best_idx = np.argmin(new_fitness)
                    tribe.best = new_population[tribe.best_idx]
                    tribe.best_fitness = new_fitness[tribe.best_idx]
        return self.tribes

    def optimize(self):
        best_fitness = min(tribe.best_fitness for tribe in self.tribes)
        best_solution = min(self.tribes, key=lambda t: t.best_fitness).best
        iter = 0
        while self.Nevals < self.maxevals:
            prob_matrix = self._marriage_probability()
            if self.Nevals>0.5*self.maxevals:
                self._marriage(prob_matrix)
            for tribe in self.tribes:
                tribe.evolve()
            #self.tribes = self._reinitialize_tribes()

            all_population = np.vstack([tribe.population for tribe in self.tribes])
            all_fitness = np.hstack([tribe.fitness for tribe in self.tribes])

            if np.min(all_fitness) < best_fitness:
                best_fitness = np.min(all_fitness)
                best_solution = all_population[np.argmin(all_fitness)]

            self.Nevals += self.popsize
            if self.Nevals >= self.maxevals:
                break
            iter= iter+1

        return OptimizeResult(x=best_solution, fun=best_fitness, nfev=self.Nevals)


# CEC 2022 test 
------


In [27]:
class FunctionEvaluator:
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += 1
        ret = self.func(np.array([X]))[0]
        #print(X.shape, ret.shape)
        return ret
    
class VectorizedEvaluator : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += X.shape[0]
        ret = self.func(np.array(X))
        return ret
    
class PyminionFunc : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X, data=None):
        X= np.array(X)
        self.n_calls += X.shape[0]
        return  self.func(X)
    
class VectorizedEvaluatorDE : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        X = np.array(X).T
        self.n_calls += X.shape[0]
        return self.func(X)
    
class TestFunction:
    def __init__(self, ID, fun, dim, global_minimum=None, inequality_constrains_num=0, equality_constraints_num=0):
        self.ID = ID
        self.fun = fun
        self.dimensionality = dim
        self.equality_constraints_num = equality_constraints_num
        self.inequality_constraints_num = inequality_constrains_num
        self.global_minimum = global_minimum

    def evaluate(self, x):
        return self.fun(x)

    def __call__(self, x):
        x=np.array(x)
        return self.fun(x)[0], np.zeros((x.shape[0], 1)), np.zeros((x.shape[0], 1))


results = []
def test_optimization(func, bounds, dimension, func_name, Nmaxeval):
    result = {}
    result['Dimensions'] = dimension
    result['Function'] = func_name
    # Initialize bounds
    bounds_list = [bounds] * dimension
    # Create wrapped function evaluator
    evaluator = FunctionEvaluator(func)
    vecEvaluator = VectorizedEvaluator(func)
    vecEvaluatorDE = VectorizedEvaluatorDE(func)
    evaluator_other = TestFunction(0, func, dimension, 0, 0, 0)
    pyminionFunc = func
     # Lshade Optimization
    popsize= 20+round(dimension/2)


    tde = tDE(vecEvaluator, bounds_list, args=(), x0=None, population_size=50, maxevals=Nmaxeval, strategy="current_to_pbest1bin", tol=0.0, seed=None, Ntribes=3)
    res = tde.optimize() 
    result["tDE"] = res.fun

    #other = j2020() 
    #res = other.optimize(evaluator_other, dimension, Nmaxeval, np.full(dimension, bounds[1]),np.full(dimension, bounds[0]) )
    #result["J2020"] = res[0].objective

    lshade = LSHADE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, 
                 strategy= "current_to_pbest1bin", relTol=0.0, minPopSize=max(round(dimension/2), 20), memeorySize=int(2*popsize), callback=None, boundStrategy="reflect-random", seed=None)
    res = lshade.optimize()
    result['LSHADE'] = res.fun

    shade = MFADE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, 
                 strategy= "current_to_pbest_A1_1bin", relTol=0.0, minPopSize=max(round(dimension/2), 20), memeorySize=int(2*popsize), callback=None, boundStrategy="reflect-random", seed=None)
    res = shade.optimize()
    result['MFADE'] = res.fun


    jade = LJADE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, 
                 strategy="current_to_pbest_A1_1bin", relTol=0.0, minPopSize=max(round(dimension/2), 20), c=0.5, callback=None, boundStrategy="reflect-random", seed=None)
    res = jade.optimize()
    result['LJADE'] = res.fun



    #sjade = SJADE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, 
    #            relTol=0.0, minPopSize=max(round(dimension/2), 5), c=0.5, callback=None, boundStrategy="reflect-random", seed=None)
    #res = sjade.optimize()
    #result['SJADE'] = res.fun
    

    #powell = Powell (pyminionFunc, bounds_list, data=None, x0=[1.5]*dimension, maxevals=Nmaxeval, relTol=0.0)
    #res = powell.optimize()
    #result['Powell'] = res.fun

    #powell = minimize(evaluator, x0=[1.5]*dimension, bounds=bounds_list, options={"maxfev":Nmaxeval,}, method="Powell")
    #result['Powell Scip'] = powell.fun

    #anm = NelderMead (pyminionFunc, bounds_list, data=None, x0=[1.5]*dimension, maxevals=Nmaxeval, relTol=0.0)
    #res = anm.optimize()
    #result['NM'] = res.fun

    #anm = minimize(evaluator, x0=[1.5]*dimension, bounds=bounds_list, options={"maxfev":Nmaxeval, "adaptive": True}, method="Nelder-Mead")
    #result['NM Scip'] = powell.fun

    #gwo = GWO_DE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, F=0.5, CR=0.7, elimination_prob=0.1, relTol=0.0001, callback=None, boundStrategy="reflect-random", seed=None)
    #res = gwo.optimize()
    #result['GWO_DE++'] = res.fun


    #Differential Evolution (DE)
    #vecEvaluatorDE.n_calls = 0
    #psize = round(max(int(150/dimension), 1)*dimension)
    #de_result = differential_evolution(vecEvaluatorDE, bounds_list, popsize=5, strategy='best1exp',
     #                                    maxiter=int(Nmaxeval/(5*dimension)), vectorized=True, updating="deferred", disp=False,polish=False)
    #result['Scipy DE'] = de_result.fun



    results.append(result)
    print(result)
    

    if False:
      plt.figure(figsize=(4,3))
      plt.plot(jade.muCR, label ="ljade CR")
      plt.plot(jade.muF, label="ljade F")
      plt.legend()
      plt.show()
  

      plt.figure(figsize=(4,3))
      plt.plot(lshade.muCR, label ="LSHADE CR")
      plt.plot(lshade.muF, label="LSHADE F")
      plt.legend()
      plt.show()

      plt.figure(figsize=(4,3))
      plt.plot(shade.muCR, label ="MFADE CR")
      plt.plot(shade.muF, label="MFADE F")
      plt.legend()
      plt.show()
     
      plt.figure(figsize=(4,3))
      plt.plot([r.fun for r in jade.history], label="Fitness LJADE")
      plt.plot([r.fun for r in shade.history], label="Fitness MFADE")
      plt.plot([r.fun for r in lshade.history], label="Fitness LSHADE")
      plt.legend()
      plt.yscale("log")
      plt.show() 
      
Nmaxeval =20000
dimensions = [   10]
for dim in dimensions:
    i=0
    for j in range(1, 11) : 
        cec_func = CEC2022Functions(function_number=j, dimension=dim)
        test_optimization(cec_func, (-100, 100), dim, "func_"+str(j), Nmaxeval)
        i=i+1 
        
        

# Results output
import pandas as pd
pd.set_option('display.float_format', '{:.12f}'.format)
results_df = pd.DataFrame(results)
results_df

{'Dimensions': 10, 'Function': 'func_1', 'tDE': 572.5815652999797, 'LSHADE': 300.00000000000006, 'MFADE': 300.0, 'LJADE': 300.0}
{'Dimensions': 10, 'Function': 'func_2', 'tDE': 406.0965305318376, 'LSHADE': 403.98657912336387, 'MFADE': 408.9161019277813, 'LJADE': 400.00159028293325}
{'Dimensions': 10, 'Function': 'func_3', 'tDE': 600.0021156793553, 'LSHADE': 600.0, 'MFADE': 600.0000000187752, 'LJADE': 600.0000000000001}
{'Dimensions': 10, 'Function': 'func_4', 'tDE': 819.9088794422782, 'LSHADE': 805.9744155499536, 'MFADE': 804.1410221978775, 'LJADE': 820.6184303345448}
{'Dimensions': 10, 'Function': 'func_5', 'tDE': 900.007327920085, 'LSHADE': 900.0, 'MFADE': 900.0, 'LJADE': 900.0}
{'Dimensions': 10, 'Function': 'func_6', 'tDE': 1832.1555121944966, 'LSHADE': 1814.6706865731094, 'MFADE': 1805.043006096268, 'LJADE': 1877.5626225965289}
{'Dimensions': 10, 'Function': 'func_7', 'tDE': 2022.3389992965867, 'LSHADE': 2000.389757321326, 'MFADE': 2009.0010030393928, 'LJADE': 2005.2747110464595}


,Dimensions,Function,tDE,LSHADE,MFADE,LJADE
0,10,func_1,572.581565299980,300.000000000000,300.000000000000,300.000000000000
1,10,func_2,406.096530531838,403.986579123364,408.916101927781,400.001590282933
2,10,func_3,600.002115679355,600.000000000000,600.000000018775,600.000000000000
3,10,func_4,819.908879442278,805.974415549954,804.141022197878,820.618430334545
4,10,func_5,900.007327920085,900.000000000000,900.000000000000,900.000000000000
5,10,func_6,1832.155512194497,1814.670686573109,1805.043006096268,1877.562622596529
6,10,func_7,2022.338999296587,2000.389757321326,2009.001003039393,2005.274711046460
7,10,func_8,2223.823812555952,2220.452678397021,2220.925179487838,2222.982871568361
8,10,func_9,2529.284382708488,2529.284382708487,2529.284382708487,2529.284382708487
9,10,func_10,2500.451288480324,2500.357499596126,2500.517451044811,2500.375653889255


-------
# Common test function
------

# 

In [5]:
function_name_to_function = {
    "sphere": sphere,
    "rosenbrock": rosenbrock,
    "rastrigin": rastrigin,
    "drop_wave":  drop_wave, 
    "griewank": griewank,
    "ackley": ackley,
    "zakharov": zakharov,
    "goldstein_price" : goldstein_price,
    "michalewicz": michalewicz,
    "easom" : easom,
    "levy": levy,
    "dixon_price": dixon_price,
    "bent_cigar": bent_cigar,
    "discus": discus,
    "weierstrass": weierstrass,
    "happy_cat": happy_cat,
    "hgbat": hgbat,
    "hcf": hcf,
    "grie_rosen": grie_rosen,
    "escaffer6": escaffer6,
    "hybrid_composition1": hybrid_composition1,
    "hybrid_composition2": hybrid_composition2,
    "hybrid_composition3": hybrid_composition3,
    "step": step,
    "quartic": quartic,
    "schaffer2": schaffer2,
    "brown": brown,
    "exponential": exponential,
    "styblinski_tang": styblinski_tang,
    "sum_squares": sum_squares
}

# Map function implementations to function names
function_to_function_name = {v: k for k, v in function_name_to_function.items()}

# Function evaluation counter
class FunctionEvaluator:
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += 1
        ret = self.func(np.array([X]))[0]
        #print(X.shape, ret.shape)
        return 1000+ret
    
class VectorizedEvaluator : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += X.shape[0]
        ret = self.func(np.array(X))
        return 1000+ret
    
class PyminionFunc : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X, data=None):
        X= np.array(X)
        self.n_calls += X.shape[0]
        return  1000+self.func(X)
    
class VectorizedEvaluatorDE : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        X = np.array(X).T
        self.n_calls += X.shape[0]
        return 1000+self.func(X)


In [6]:
results = []
def test_optimization(func, bounds, dimension, func_name, Nmaxeval):
    result = {}
    result['Dimensions'] = dimension
    result['Function'] = func_name
    # Initialize bounds
    bounds_list = [bounds] * dimension
    # Create wrapped function evaluator
    evaluator = FunctionEvaluator(func)
    vecEvaluator = VectorizedEvaluator(func)
    vecEvaluatorDE = VectorizedEvaluatorDE(func)
    pyminionFunc = PyminionFunc(func)
     # Lshade Optimization
    popsize=20+round(dimension)

    lshade = LSHADE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, 
                 strategy= "current_to_pbest1bin", relTol=0.0, minPopSize=max(round(dimension/2), 10), memeorySize=2*popsize, callback=None, boundStrategy="reflect-random", seed=None)
    res = lshade.optimize()
    result['LSHADE'] = res.fun


    shade = MFADE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, 
                 strategy= "current_to_pbest1bin", relTol=0.0, minPopSize=max(round(dimension/2), 10), memeorySize=3*popsize, callback=None, boundStrategy="reflect-random", seed=None)
    res = shade.optimize()
    result['MFADE'] = res.fun


    jade = LJADE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, 
                 strategy="current_to_pbest1bin", relTol=0.0, minPopSize=max(round(dimension/2), 10), c=0.5, callback=None, boundStrategy="reflect-random", seed=None)
    res = jade.optimize()
    result['FADE'] = res.fun

    #powell = Powell (pyminionFunc, bounds_list, data=None, x0=[1.5]*dimension, maxevals=Nmaxeval, relTol=0.0)
    #res = powell.optimize()
    #result['Powell'] = res.fun

    #powell = minimize(evaluator, x0=[1.5]*dimension, bounds=bounds_list, options={"maxfev":Nmaxeval,}, method="Powell")
    #result['Powell Scip'] = powell.fun

    #anm = NelderMead (pyminionFunc, bounds_list, data=None, x0=[1.5]*dimension, maxevals=Nmaxeval, relTol=0.0)
    #res = anm.optimize()
    #result['NM'] = res.fun

    #anm = minimize(evaluator, x0=[1.5]*dimension, bounds=bounds_list, options={"maxfev":Nmaxeval, "adaptive": True}, method="Nelder-Mead")
    #result['NM Scip'] = powell.fun

    #gwo = GWO_DE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, F=0.5, CR=0.7, elimination_prob=0.1, relTol=0.0001, callback=None, boundStrategy="reflect-random", seed=None)
    #res = gwo.optimize()
    #result['GWO_DE++'] = res.fun


    #Differential Evolution (DE)
    vecEvaluatorDE.n_calls = 0
    psize = round(max(int(150/dimension), 1)*dimension)
    de_result = differential_evolution(vecEvaluatorDE, bounds_list, popsize=5, strategy='best1exp',
                                         maxiter=int(Nmaxeval/(5*dimension)), vectorized=True, updating="deferred", disp=False,polish=False)
    result['Scipy DE'] = de_result.fun


    results.append(result)
    print(result)
    

    if False:
      plt.figure(figsize=(4,3))
      plt.plot(jade.muCR, label ="FADE CR")
      plt.plot(jade.muF, label="FADE F")
      plt.legend()
      plt.show()
  

      plt.figure(figsize=(4,3))
      plt.plot(shade.muCR, label ="MFADE CR")
      plt.plot(shade.muF, label="MFADE F")
      plt.legend()
      plt.show()
     
      plt.plot([r.fun for r in jade.history], label="Fitness FADE")
      plt.plot([r.fun for r in shade.history], label="Fitness MFADE")
      plt.legend()
      plt.yscale("log")
      plt.show() 
      
Nmaxeval = 10000
dimensions = [ 20]
test = function_name_to_function
#test = {"Any" : rastrigin}
for dim in dimensions:
    i=0
    for funcname, func in test.items() : #function_name_to_function.items():
        test_optimization(func, (-100, 100), dim, funcname, Nmaxeval)
        i=i+1 
        #if i>15: break 

# Results output
import pandas as pd
pd.set_option('display.float_format', '{:.12f}'.format)
results_df = pd.DataFrame(results)
results_df

{'Dimensions': 20, 'Function': 'sphere', 'LSHADE': 1000.0000000000001, 'MFADE': 1000.000000028702, 'FADE': 1000.0000000006962, 'Scipy DE': 1126.8800476190163}
{'Dimensions': 20, 'Function': 'rosenbrock', 'LSHADE': 1069.5146799760425, 'MFADE': 1061.257607936527, 'FADE': 1013.852828567996, 'Scipy DE': 631511.5986185812}
{'Dimensions': 20, 'Function': 'rastrigin', 'LSHADE': 1012.9809480841009, 'MFADE': 1024.313011493204, 'FADE': 1029.0169379920542, 'Scipy DE': 1387.0801761284204}
{'Dimensions': 20, 'Function': 'drop_wave', 'LSHADE': 999.0, 'MFADE': 999.0000000073155, 'FADE': 999.0000000000066, 'Scipy DE': 999.9138254169908}
{'Dimensions': 20, 'Function': 'griewank', 'LSHADE': 1000.0000000000001, 'MFADE': 1000.0000019311108, 'FADE': 1000.0000000006256, 'Scipy DE': 1008.6823400690392}
{'Dimensions': 20, 'Function': 'ackley', 'LSHADE': 1020.0195465599079, 'MFADE': 1020.0772604955725, 'FADE': 1020.0929279919274, 'Scipy DE': 1021.2706366477543}
{'Dimensions': 20, 'Function': 'zakharov', 'LSHAD

e:\Minion\examples\..\pyminion\test_functions.py:126: RuntimeWarning: overflow encountered in power
  result = np.sum((X[:, :-1]**2)**(X[:, 1:]**2 + 1) + (X[:, 1:]**2)**(X[:, :-1]**2 + 1), axis=1)


{'Dimensions': 20, 'Function': 'brown', 'LSHADE': inf, 'MFADE': inf, 'FADE': inf, 'Scipy DE': inf}
{'Dimensions': 20, 'Function': 'exponential', 'LSHADE': 1000.0, 'MFADE': 1000.0, 'FADE': 1000.0, 'Scipy DE': 1000.0}
{'Dimensions': 20, 'Function': 'styblinski_tang', 'LSHADE': 315.6337192639842, 'MFADE': 315.63373632553294, 'FADE': 218.40063552410402, 'Scipy DE': 6492.705300065684}
{'Dimensions': 20, 'Function': 'sum_squares', 'LSHADE': 1000.0, 'MFADE': 1000.0000033095106, 'FADE': 1000.000000025957, 'Scipy DE': 2365.7802772846344}


,Dimensions,Function,LSHADE,MFADE,FADE,Scipy DE
0,20,sphere,1000.000000000000,1000.000000028702,1000.000000000696,1126.880047619016
1,20,rosenbrock,1069.514679976043,1061.257607936527,1013.852828567996,631511.598618581193
2,20,rastrigin,1012.980948084101,1024.313011493204,1029.016937992054,1387.080176128420
3,20,drop_wave,999.000000000000,999.000000007316,999.000000000007,999.913825416991
4,20,griewank,1000.000000000000,1000.000001931111,1000.000000000626,1008.682340069039
5,20,ackley,1020.019546559908,1020.077260495573,1020.092927991927,1021.270636647754
6,20,zakharov,1993.926394627986,1077.239531192711,25981.895972535731,32014.151338989312
7,20,goldstein_price,1003.000000000000,1003.000000000000,1003.000000000000,1003.314638918545
8,20,michalewicz,986.865825925935,989.116815353247,988.581913164580,996.102615447604
9,20,easom,999.991606408826,999.994312187937,999.997101811924,1000.000000000000
